In [34]:
import pandas as pd
import numpy as np
import df_build as build
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.metrics import confusion_matrix
import epl_analysis
from collections import defaultdict

In [35]:
def rf_model(df):
    bal_rf = BalancedRandomForestClassifier()
    model_data = df.copy()
    # model prep
    game_hist = '2019-12-31'
    # construct target feature
    y = model_data.pop('team_reds')
    y = y[y.index < game_hist]
    y.shift();
    y_train, y_test = y[50:-2], y[:50]
    # construct model data
    X = model_data.drop(['opponent', 'season_num'], axis=1)
    home_flag = X.pop('home_advan')
    X = X.sort_index().rolling(3).mean()
    X = X.sort_index()[::-1]
    X.insert(0, 'home_advan', home_flag)
    X_train, X_test = X[50:-2], X[:50]
    bal_rf.fit(X_train, y_train)
    # metrics
    oob_score = bal_rf.score(X_test, y_test)
    y_predict = bal_rf.predict(X_test)
    conf_mtrx = confusion_matrix(y_test, y_predict)
    
    return oob_score, conf_mtrx

In [36]:
# every team in the coming season with data avail in csv's are in either list
# team in the list that fail first are put into fall into second list
season_20_21_team_options = ['Arsenal', 'Aston Villa', 'Burnley',
    'Chelsea', 'Crystal Palace', 'Everton', 'Fulham', 'Leicester', 'Liverpool',
    'Man City', 'Man United', 'Newcastle', 'Tottenham', 'West Brom',
    'West Ham', 'Wolves']
broken_teams = ['Brighton','Southhampton']

In [37]:
# ctl_team, team = 'Man City', 'Chelsea'

In [38]:
# team_focus = build.df_team_focus(build.csv_to_df(), team)
# team_focus = build.df_team_focus(build.csv_to_df(), ctl_team)

## Modeling

In [39]:
# score, mtrx = rf_model(team_focus)
# score, mtrx

In [40]:
accuracy_d = defaultdict(float)
for team in season_20_21_team_options:
    score = rf_model(build.df_team_focus(build.csv_to_df(), team))[0]
    accuracy_d[team] = score

/Users/zschulzb/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/Users/zschulzb/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/Users/zschulzb/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

In [41]:
values = list(accuracy_d.values())
print('Predictive Accuracy By Team In Upcoming Season')
for k, v in accuracy_d.items():
    print('{0}: {1}'.format(k, v))

print('Avg. Accuracy: {0}'.format(np.sum(values) / len(values)))

Predictive Accuracy By Team In Upcoming Season
Arsenal: 0.22
Aston Villa: 0.22
Burnley: 0.7
Chelsea: 0.7
Crystal Palace: 0.76
Everton: 0.64
Fulham: 0.68
Leicester: 0.36
Liverpool: 0.88
Man City: 0.84
Man United: 0.52
Newcastle: 0.38
Tottenham: 0.62
West Brom: 0.8
West Ham: 0.44
Wolves: 0.66
Avg. Accuracy: 0.58875


In [42]:
teams_above_50 = []

for k, v in accuracy_d.items():
    if v > 0.5:
        teams_above_50.append(k)

print('Percentage of team predicting above 50%: \n\t{0}%'.format(len(teams_above_50)/len(season_20_21_team_options)*100))

Percentage of team predicting above 50%: 
	68.75%


In [27]:
len(teams_above_50)

10